# tBLASTn in arthropods

In [2]:
import time
import os

In [3]:
clustered_seq = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-CD-HIT-updated.clstr"
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER-updated.fasta"

Extracting representative IDs from each cluster.

In [4]:
with open(clustered_seq, "r") as clusters:
    c = clusters.readlines()
rep_ID_lines = [e for e in c if e.endswith("*\n")]
rep_IDs = [e.split(", >")[1].split("...")[0] for e in rep_ID_lines]
rep_IDs

['QPN36968.1',
 'UQT02520.1',
 'QMP82367.1',
 'UVF62194.1',
 'QMP82195.1',
 'QMP82285.1',
 'QMP82388.1',
 'QMP82402.1',
 'QMP82403.1',
 'QMP82112.1',
 'QMP82168.1',
 'QMP82344.1',
 'QMP82362.1',
 'QMP82338.1',
 'QPZ88437.1',
 'UMO75733.1',
 'QED21504.1',
 'UPT53723.1',
 'AVR52572.1',
 'QMP82355.1',
 'AVR52567.1',
 'UUG74181.1',
 'UMO75734.1',
 'AJG39094.1',
 'QMP82147.1',
 'QPL15381.1',
 'QMP82248.1',
 'UMO75726.1',
 'QMP82204.1',
 'QPL15288.1',
 'QPL15340.1',
 'UMO75713.1',
 'UUG74233.1',
 'QPL15368.1',
 'UMO75705.1',
 'UMO75722.1',
 'QMP82272.1',
 'URQ09139.1',
 'QIJ70032.1',
 'QMP82229.1',
 'QWC36487.1',
 'UUG74184.1',
 'AJG39084.1',
 'UMO75717.1',
 'DAZ85673.1',
 'AJG39092.1',
 'AJG39088.1',
 'QDK54862.1',
 'QEM39320.1',
 'AJG39089.1',
 'QPZ88434.1',
 'AJG39086.1',
 'QKK82920.1',
 'QPL15369.1',
 'QPZ88431.1',
 'UUG74234.1',
 'ACY56282.1',
 'UMO75729.1',
 'UYL95486.1',
 'ASR92124.1',
 'UUT42596.1',
 'QPN36942.1',
 'URY50684.1',
 'UUT42625.1',
 'AIY25030.1',
 'QOE76794.1',
 'DAZ91106

In [6]:
len(rep_IDs)

109

Moving selected sequences to a new directory.

In [8]:
import shutil

source_folder = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1/"
destination_folder = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/PB1-clusters/"

for file in rep_IDs:
    source = source_folder + f"{file}.fasta"
    destination = destination_folder + f"{file}.fasta"
    shutil.copy(source, destination)

# Building nt database for local BLAST

If the database is alreade built, skip to [tBLASTn](#running-local-blast).

Following https://bioinf.shenwei.me/taxonkit/tutorial/#extract-all-protein-sequences-of-specific-taxons-from-the-ncbi-nr-database tutorial

Steps:

### 1. Setting up taxonkit:
1.1 Downloаd taxonkit

1.2 Downloading and decompressing taxdump.tar.gz: ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz (as recommended https://bioinf.shenwei.me/taxonkit/download/)


In [ ]:
%%bash
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz' --output ./taxdump.tar.gz

In [ ]:
%%bash
tar -zxvf taxdump.tar.gz

1.3 Copying some of the taxdump files to `$HOME/.taxonkit` directory.

In [ ]:
%%bash
cp names.dmp nodes.dmp delnodes.dmp merged.dmp $HOME/.taxonkit

### 2. Writing Arthropoda IDs to text file.

In [ ]:
%%bash
id=6656
taxonkit list --ids $id --indent "" > $id.taxid.txt
wc -l $id.taxid.txt

  925155 6656.taxid.txt


### 3. Retrieving target accessions:

3.1 Downloading `nucl_gb.accession2taxid.gz` file for nt database (or `prot.accession2taxid.gz` for nr db) from NCBI server: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/

Note: specify the name of the output file in curl command (the name of directory isn't enough).

In [ ]:
%%bash
# nt database
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz' --output /Volumes/WD_BLACK/Brigita/nt/nucl_gb.accession2taxid.gz

In [ ]:
%%bash
# nr database
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz' --output /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz

3.2 Retrieving target accessions (taxid -> accession numbers).

In [4]:
%%bash
# nt database
source ~/.bashrc
pigz -dc /Volumes/WD_BLACK/Brigita/nt/nucl_gb.accession2taxid.gz \
    | csvtk grep -t -f taxid -P /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.txids \
    | csvtk cut -t -f accession.version,taxid \
    | sed 1d \
    > /Volumes/WD_BLACK/Brigita/nt/6656.acc2taxid.txt

cut -f 1 /Volumes/WD_BLACK/Brigita/nt/6656.acc2taxid.txt > /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt

wc -l /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt

 34241660 /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt


In [ ]:
%%bash
# nr database
pigz -dc /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz \
    | csvtk grep -t -f taxid -P /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.taxid.txt \
    | csvtk cut -t -f accession.version,taxid \
    | sed 1d \
    > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt

cut -f 1 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

wc -l /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

 19035895 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt


### 4. Retrieving FASTA sequences from pre-formated blastdb.

4.1 Downloading databases:

In [ ]:
%%bash 
# nt database
curl -O "ftp://ftp.ncbi.nih.gov/blast/db/nt_euk.[00-100].tar.gz" --output-dir /Volumes/WD_BLACK/Brigita/nt/pre_nt

In [ ]:
%%bash
ls nt_euk.*.tar.gz | xargs -I % tar -zxv -f % -C /Volumes/WD_BLACK/Brigita/nt/extracted

In [ ]:
%%bash
# nr database
curl -O "ftp://ftp.ncbi.nih.gov/blast/db/nr.[00-100].tar.gz" --output-dir /Volumes/EXTERNAL/Orthomyxoviridae/blast

In [ ]:
%%bash
ls nr.*.tar.gz | xargs -I % tar -zxv -f % -C /Volumes/EXTERNAL/Orthomyxoviridae/blast/extracted/

Exporting nt_euk.fa (nr.fa) from pre-formated blastdb. 

In [ ]:
%%bash
blastdbcmd -db /Volumes/WD_BLACK/Brigita/nt/extracted/nt_euk -dbtype nucl -entry all -outfmt "%f" -out - | pigz -c > /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.fa.gz

In [ ]:
%%bash
cd /Volumes/EXTERNAL/Orthomyxoviridae/blast/nr
blastdbcmd -db nr -dbtype prot -entry all -outfmt "%f" -out - | pigz -c > /Volumes/EXTERNAL/Orthomyxoviridae/nr.fa.gz

Installing seqkit.

The script below was written as a text file and run from a command line: bash filter-sequence.sh 

Extracting *Arthropoda* sequences from the entire database.

In [ ]:
%%bash
# nt database
time cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.fa.gz) \
    | perl -e 'BEGIN{ $/ = "\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\W+//; print ">$h1\n$s";} } ' \
    | seqkit grep -f /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa.gz

In [ ]:
%%bash
#nr database
time cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.fa.gz) \
    | perl -e 'BEGIN{ $/ = "\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\W+//; print ">$h1\n$s";} } ' \
    | seqkit grep -f /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz

Counting sequences.

In [ ]:
%%bash
wc -l /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt
pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz | grep '^>' -c

10746263


Formatting BLAST database.

In [ ]:
%%bash
# nt database
pigz -dc /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa.gz > /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa

In [ ]:
%%bash
# nt database
makeblastdb -parse_seqids -in /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa -dbtype nucl -out /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656

Output:

>Building a new DB, current time: 09/06/2023 17:52:16\
>New DB name:   /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656\
>New DB title:  /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa\
>Sequence type: Nucleotide\
>Keep MBits: T\
>Maximum file size: 3000000000B\
>Adding sequences from FASTA; added 17068454 sequences in 9733.49 seconds.

In [9]:
%%bash
# nr database
pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz > /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa

In [10]:
%%bash
# nr database
makeblastdb -parse_seqids -in /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa -dbtype prot -out /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656



Building a new DB, current time: 08/16/2023 11:22:29
New DB name:   /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656
New DB title:  /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 10746263 sequences in 509.398 seconds.




# Running local BLAST.

Database (-db): nt/6656/nt_euk.6656\
E-value (-evalue): 1e-6\
CPUs (-num_threads): 8\
Number of hits (-max_target_seqs): 1000\
Output format (-outfmt): 5 (output is written in XML)

In [ ]:
%%bash
# BLASTing 20 sequences
time tblastn -db nt/6656/nt_euk.6656 -query PB1-20seq.fasta -out PB1-20tblastn.xml -evalue 1e-6 -num_threads 8 -max_target_seqs 1000 -outfmt 5

In [ ]:
%%bash
#test run
tblastn -db /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656 -query /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/blast-test.fasta -out /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/blast-result2 -evalue 1e-6 -num_threads 4 -max_target_seqs 1000 -outfmt 5

Another way to download nt database:

In [ ]:
%%bash 
update_blastdb.pl --decompress nt

# Analyzing hits

In [1]:
import pandas as pd
import numpy as np
from Bio import SearchIO

Exploring data (can be deleted later).

In [41]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
hits_total = []
for query in BLAST_results_20:
    print(query.id)
    hits = []
    for hit in query:
        hits.append(hit.id)
    hits_total.append(len(hits))
    print(f"Number of hits: ", len(hits))
    print("--------------------------")

print(sum(hits_total))

AAA42968.1
Number of hits:  259
--------------------------
AAC25959.1
Number of hits:  249
--------------------------
AAL67962.1
Number of hits:  13
--------------------------
AAU94857.1
Number of hits:  306
--------------------------
ACY56282.1
Number of hits:  640
--------------------------
AED98371.1
Number of hits:  262
--------------------------
AIY25030.1
Number of hits:  630
--------------------------
AJG39084.1
Number of hits:  646
--------------------------
AJG39085.1
Number of hits:  588
--------------------------
AJG39086.1
Number of hits:  672
--------------------------
AJG39088.1
Number of hits:  673
--------------------------
AJG39089.1
Number of hits:  672
--------------------------
AJG39092.1
Number of hits:  651
--------------------------
AJG39094.1
Number of hits:  646
--------------------------
AJG39096.1
Number of hits:  685
--------------------------
ALJ78713.1
Number of hits:  268
--------------------------
APP91612.1
Number of hits:  269
-------------------------

In [42]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
count = 0
for query in BLAST_results_20:
    print(query.id)
    for hit in query:
        count += 1
        hsps = []
        for hsp in hit:
            hsps.append(hsp)
        print(f"{count} Number of HSPs for {hit.id}: ", len(hsps))    
    print("--------------------------")

AAA42968.1
1 Number of HSPs for emb|OW052077.1|:  16
2 Number of HSPs for emb|OX421934.1|:  5
3 Number of HSPs for emb|OX411715.1|:  8
4 Number of HSPs for emb|OX382229.1|:  3
5 Number of HSPs for emb|OU342663.1|:  1
6 Number of HSPs for emb|OY292434.1|:  1
7 Number of HSPs for emb|LR990212.1|:  2
8 Number of HSPs for emb|OV656855.1|:  4
9 Number of HSPs for emb|OV121058.1|:  2
10 Number of HSPs for emb|FR990001.1|:  7
11 Number of HSPs for emb|OU744810.1|:  10
12 Number of HSPs for emb|OW388277.2|:  2
13 Number of HSPs for emb|OU426947.1|:  1
14 Number of HSPs for emb|OX424458.1|:  4
15 Number of HSPs for emb|OU026114.1|:  2
16 Number of HSPs for emb|OU974029.1|:  1
17 Number of HSPs for emb|OX419599.1|:  2
18 Number of HSPs for emb|OX421465.1|:  1
19 Number of HSPs for emb|OX382355.1|:  2
20 Number of HSPs for emb|OW203721.1|:  3
21 Number of HSPs for emb|OY560206.1|:  1
22 Number of HSPs for emb|OW971904.1|:  1
23 Number of HSPs for emb|OW052072.1|:  1
24 Number of HSPs for emb|LR99

### **Extracting queries and hits from BLAST XML files.**

Creating a list with accession numbers of queries (viral PB1 sequences).

In [25]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")

PB1_20 = [query.id for query in BLAST_results_20]
PB1_89 = [query.id for query in BLAST_results_89]
PB1 = PB1_20 + PB1_89

print(len(PB1))

109


Extracting all hits from XML files.

In [26]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
hits_20 = []
for query in BLAST_results_20:
    hits = []
    for hit in query:
        hit_id = hit.id.split("|")[1]
        hits.append(hit_id)
    hits_20.append(hits)
print(hits_20)

[['OW052077.1', 'OX421934.1', 'OX411715.1', 'OX382229.1', 'OU342663.1', 'OY292434.1', 'LR990212.1', 'OV656855.1', 'OV121058.1', 'FR990001.1', 'OU744810.1', 'OW388277.2', 'OU426947.1', 'OX424458.1', 'OU026114.1', 'OU974029.1', 'OX419599.1', 'OX421465.1', 'OX382355.1', 'OW203721.1', 'OY560206.1', 'OW971904.1', 'OW052072.1', 'LR990280.1', 'OY039093.1', 'OU612012.1', 'OX243865.1', 'OX402555.1', 'OX387918.1', 'OU963894.1', 'OY365787.1', 'OX419725.1', 'OX465245.1', 'OX421890.1', 'OU745266.1', 'OU452198.1', 'OX374634.1', 'OU342464.1', 'OU612081.1', 'LR990192.1', 'OX101744.1', 'OW443294.2', 'LR991043.1', 'OX387434.1', 'OX384531.1', 'OY294033.1', 'OW052003.1', 'OX419582.1', 'OX335686.1', 'OX421221.1', 'OX276440.1', 'OX376307.1', 'LR990246.1', 'OV281311.1', 'OX392509.1', 'OW285216.1', 'LR990971.1', 'OX438886.1', 'OX465478.1', 'OX376305.1', 'LR990852.1', 'OU612038.1', 'OX382357.1', 'OU785243.1', 'OY039083.1', 'OW121784.1', 'OU342482.1', 'OX421869.1', 'OU426919.1', 'OU015575.1', 'OU343118.1', 'OU0

In [86]:
len(hits_20)

20

In [27]:
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")
hits_89 = []
for query in BLAST_results_89:
    hits = []
    for hit in query:
        hit_id = hit.id.split("|")[1]
        hits.append(hit_id)
    hits_89.append(hits)
print(hits_89)

[['LR990212.1', 'OX421890.1', 'OX376305.1', 'OU452198.1', 'OU342663.1', 'OU015512.1', 'OW052072.1', 'OU426932.1', 'OU974029.1', 'OX384531.1', 'OX382355.1', 'OX387918.1', 'OX101721.1', 'OY294061.1', 'OX465245.1', 'OU015575.1', 'OX387434.1', 'OX421397.1', 'OU612081.1', 'OU026083.1', 'OU426919.1', 'OW052077.1', 'OX374634.1', 'OW285216.1', 'OW964569.2', 'OX637540.1', 'OX243865.1', 'OY292434.1', 'OD584463.1', 'OX382229.1', 'OU342954.1', 'OX419599.1', 'OX421934.1', 'OX411715.1', 'OY720162.1', 'OX387435.1', 'OX369255.1', 'OX421465.1', 'OU343118.1', 'OU342464.1', 'OW203987.1', 'OU426947.1', 'OW388147.1', 'LR990192.1', 'OX276440.1', 'OX384041.1', 'OY560206.1', 'OU785243.1', 'OU342946.1', 'OW203721.1', 'OX376307.1', 'OW052003.1', 'OU026114.1', 'OX457144.1', 'OV121058.1', 'OU744810.1', 'OY293313.1', 'OV656855.1', 'OX419742.1', 'OW964551.2', 'OX442329.1', 'OY039093.1', 'OX411315.1', 'OX419582.1', 'OX421463.1', 'OU342482.1', 'OE843393.1', 'OX101744.1', 'OU538821.1', 'LR990971.1', 'FR990001.1', 'OX3

In [96]:
print(len(hits_89))

89


In [28]:
hits = hits_20 + hits_89
print(len(hits))

109


Writing query IDs into a PB1 list.

In [29]:
PB1_repeats = []

for i in range(len(PB1)):
    repeats = len(hits[i])
    for r in range(repeats):
        PB1_repeats.append(PB1[i])

PB1_repeats

['AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968.1',
 'AAA42968

In [107]:
len(PB1_repeats)

56188

Unnesting hits list.

In [30]:
HITS = sum(hits, [])
print(HITS)

['OW052077.1', 'OX421934.1', 'OX411715.1', 'OX382229.1', 'OU342663.1', 'OY292434.1', 'LR990212.1', 'OV656855.1', 'OV121058.1', 'FR990001.1', 'OU744810.1', 'OW388277.2', 'OU426947.1', 'OX424458.1', 'OU026114.1', 'OU974029.1', 'OX419599.1', 'OX421465.1', 'OX382355.1', 'OW203721.1', 'OY560206.1', 'OW971904.1', 'OW052072.1', 'LR990280.1', 'OY039093.1', 'OU612012.1', 'OX243865.1', 'OX402555.1', 'OX387918.1', 'OU963894.1', 'OY365787.1', 'OX419725.1', 'OX465245.1', 'OX421890.1', 'OU745266.1', 'OU452198.1', 'OX374634.1', 'OU342464.1', 'OU612081.1', 'LR990192.1', 'OX101744.1', 'OW443294.2', 'LR991043.1', 'OX387434.1', 'OX384531.1', 'OY294033.1', 'OW052003.1', 'OX419582.1', 'OX335686.1', 'OX421221.1', 'OX276440.1', 'OX376307.1', 'LR990246.1', 'OV281311.1', 'OX392509.1', 'OW285216.1', 'LR990971.1', 'OX438886.1', 'OX465478.1', 'OX376305.1', 'LR990852.1', 'OU612038.1', 'OX382357.1', 'OU785243.1', 'OY039083.1', 'OW121784.1', 'OU342482.1', 'OX421869.1', 'OU426919.1', 'OU015575.1', 'OU343118.1', 'OU02

In [109]:
print(len(HITS))

56188


Unique sequences within HITS list.

In [110]:
print(len(set(HITS)))

857


### **Extracting coordinates of hits from BLAST XML files.**

In [125]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
#count = 0
for query in BLAST_results_20:
    print(query.id)
    for hit in query:
        #count += 1
        #hsps = []
        for hsp in hit:
            print(hsp.hit_range)
            #hsps.append(hsp)
        #print(f"{count} Number of HSPs for {hit.id}: ", len(hsps))    
    print("--------------------------")

AAA42968.1
(3591387, 3593460)
(3596636, 3598655)
(3584067, 3586140)
(3606246, 3608319)
(3608686, 3610759)
(3611126, 3613199)
(3603806, 3605879)
(3616006, 3618079)
(3618446, 3620519)
(3586507, 3588580)
(3613566, 3615639)
(3588947, 3591020)
(3593827, 3595846)
(3581626, 3583420)
(3583427, 3583700)
(1714994, 1716752)
(5542508, 5544656)
(5546387, 5548532)
(5539153, 5540677)
(20041023, 20042589)
(19983300, 19984866)
(25339995, 25342158)
(25343908, 25346071)
(25359560, 25361723)
(25351734, 25353897)
(25355647, 25357810)
(25363473, 25365636)
(25336082, 25338245)
(25347821, 25349984)
(15016539, 15018585)
(15053393, 15055439)
(14977488, 14979036)
(15939179, 15940949)
(12671819, 12673874)
(7770762, 7772736)
(8174108, 8176097)
(5957675, 5959418)
(6261933, 6263676)
(6258972, 6259815)
(6267378, 6268314)
(8132573, 8134388)
(8134458, 8134698)
(2542105, 2543767)
(2535016, 2535832)
(2519780, 2520596)
(2527404, 2528220)
(2528443, 2529064)
(2520819, 2521440)
(2536055, 2536676)
(3660040, 3661774)
(3668088,

Wrinting min start and max end coordinates (or max start and min end if the sequence is reversed) of each hit in Hit_start and Hit_end lists.

In [31]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")

#ar yra būdas sutrumpinti kodą? -> parašyti funkciją ir ją pritaikyti kiekvienam naujam failui

Hit_start = []
Hit_end = []

for query in BLAST_results_20:
    for hit in query:
        hsp_start = [hsp.hit_start for hsp in hit if hsp.hit_start < hsp.hit_end]
        hsp_end = [hsp.hit_end for hsp in hit if hsp.hit_start < hsp.hit_end]
        if hsp_start < hsp_end:
            Hit_start.append(min(hsp_start))
            Hit_end.append(max(hsp_end))
        hsp_start = [hsp.hit_start for hsp in hit if hsp.hit_start > hsp.hit_end]
        hsp_end = [hsp.hit_end for hsp in hit if hsp.hit_start > hsp.hit_end]
        if hsp_start > hsp_end:
            Hit_start.append(max(hsp_start))
            Hit_end.append(min(hsp_end))

for query in BLAST_results_89:
    for hit in query:
        hsp_start = [hsp.hit_start for hsp in hit if hsp.hit_start < hsp.hit_end]
        hsp_end = [hsp.hit_end for hsp in hit if hsp.hit_start < hsp.hit_end]
        if hsp_start < hsp_end:
            Hit_start.append(min(hsp_start))
            Hit_end.append(max(hsp_end))
        hsp_start = [hsp.hit_start for hsp in hit if hsp.hit_start > hsp.hit_end]
        hsp_end = [hsp.hit_end for hsp in hit if hsp.hit_start > hsp.hit_end]
        if hsp_start > hsp_end:
            Hit_start.append(max(hsp_start))
            Hit_end.append(min(hsp_end))


In [139]:
print(len(Hit_start))
print(len(Hit_end))

56188
56188


Finding number of HSPs per hit.

In [23]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")

hsp_number = []

for query in BLAST_results_20:
    for hit in query:
        hsp_number.append(len(hit.hsps))

for query in BLAST_results_89:
    for hit in query:
        hsp_number.append(len(hit.hsps))

In [24]:
print(len(hsp_number))

56188


Finding mean HSP hit length per hit and standard deviation.

In [22]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")

hsp_mean_length = [] # HSP hit mean length
hsp_sd = [] # standard deviation of the above

for query in BLAST_results_20:
    for hit in query:
        hit_spans = []
        for hsp in hit:
            hit_spans.append(hsp.hit_span)
        
        hsp_mean_length.append(np.mean(hit_spans))
        hsp_sd.append(np.std(hit_spans))

for query in BLAST_results_89:
    for hit in query:
        hit_spans = []
        for hsp in hit:
            hit_spans.append(hsp.hit_span)
        
        hsp_mean_length.append(np.mean(hit_spans))
        hsp_sd.append(np.std(hit_spans))

print(len(hsp_mean_length), len(hsp_sd))

56188 56188


Creating a dataframe with hits for each query.

In [38]:
PB1hitsDF = pd.DataFrame(list(zip(PB1_repeats, HITS)), columns = ["PB1", "Hits"])

In [39]:
PB1hitsDF

,PB1,Hits
0,AAA42968.1,OW052077.1
1,AAA42968.1,OX421934.1
2,AAA42968.1,OX411715.1
3,AAA42968.1,OX382229.1
4,AAA42968.1,OU342663.1
...,...,...
56183,UYX69296.1,OX467285.1
56184,UYX69296.1,OU426941.1
56185,UYX69296.1,HG992236.1
56186,UYX69296.1,OW028721.1


In [40]:
PB1hitsDF["Hit_start"] = Hit_start
PB1hitsDF["Hit_end"] = Hit_end
PB1hitsDF["Number_of_HSPs"] = hsp_number
PB1hitsDF["HSP_mean_length"] = hsp_mean_length
PB1hitsDF["HSP_sd"] = hsp_sd

In [41]:
PB1hitsDF

,PB1,Hits,Hit_start,Hit_end,Number_of_HSPs,HSP_mean_length,HSP_sd
0,AAA42968.1,OW052077.1,1714994,3620519,16,1916.625,435.290690
1,AAA42968.1,OX421934.1,5539153,20042589,5,1789.800,291.649379
2,AAA42968.1,OX411715.1,25336082,25365636,8,2163.000,0.000000
3,AAA42968.1,OX382229.1,14977488,15055439,3,1880.000,234.759451
4,AAA42968.1,OU342663.1,15939179,15940949,1,1770.000,0.000000
...,...,...,...,...,...,...,...
56183,UYX69296.1,OX467285.1,40398442,40399256,2,414.000,114.000000
56184,UYX69296.1,OU426941.1,8943832,8946011,2,420.000,0.000000
56185,UYX69296.1,HG992236.1,619229,619814,1,585.000,0.000000
56186,UYX69296.1,OW028721.1,7452498,7453062,1,564.000,0.000000


Creating a file with hit IDs and genome coordinates of HSPs that fit into the length [1000; 2400].

In [52]:
import os
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
#BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")

path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/orthomyxo"
with open(os.path.join(path, "PB1-HSPs.txt"), "a") as file:
    for query in BLAST_results_20:
        for hit in query:
            hit_id = hit.id.split("|")[1]
            for hsp in hit:
                if (hsp.hit_span > 1000) and (hsp.hit_span < 2400):
                    line = f"{hit_id} {hsp.hit_start} {hsp.hit_end} \n"   
                    file.write(line)


In [55]:
path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/orthomyxo"
with open(os.path.join(path, "PB1-HSPs.txt"), "r") as file:
    for count, line in enumerate(file):
        pass
print('Total Lines', count + 1)

Total Lines 29296


In [54]:
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")
path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/orthomyxo"
with open(os.path.join(path, "PB1-HSPs.txt"), "a") as file:
    for query in BLAST_results_89:
        for hit in query:
            hit_id = hit.id.split("|")[1]
            for hsp in hit:
                if (hsp.hit_span > 1000) and (hsp.hit_span < 2400):
                    line = f"{hit_id} {hsp.hit_start} {hsp.hit_end} \n"   
                    file.write(line)

Downloading arthropoda genomic sequences at coordinates from PB1-HSPs.txt file.

In [63]:
import os
from Bio import SeqIO
from Bio import Entrez

path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/orthomyxo"

handle = Entrez.efetch(db = "nuccore", id = "OW052077.1", seq_start = 3591387, seq_stop = 3593460, rettype = 'gb')
record = SeqIO.read(handle, "genbank")

with open(os.path.join(path, f"{record.id}.gb"), "w") as file:
    SeqIO.write(record, f"{record.id}.gb", 'gb')
          

In [ ]:
import os
from Bio import SeqIO
from Bio import Entrez

path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/orthomyxo"

Entrez.email = "brygida.serebriakova@gmail.com"

with open(os.path.join(path, "PB1-HSPs.txt"), "r") as file:
    for line in file:
        try:
            coordinates = line.split(" ")
            handle = Entrez.efetch(db = "nuccore", id = coordinates[0], seq_start = coordinates[1], seq_stop = coordinates[2], rettype = 'gb')
            record = SeqIO.read(handle, "genbank")
            with open(os.path.join(path, f"{record.id}.gb"), "w") as file:
                SeqIO.write(record, f"{record.id}.gb", 'gb')
        except:
            time.sleep(60)
